# Lab: AI-Powered Tech News Assistant

## What You'll Build

An intelligent news assistant that fetches tech and AI news based on natural questions.

**Example:**
```
You: "What's new in AI today?"
AI: [Fetches headlines] "Here are today's top AI stories..."

You: "Find articles about ChatGPT"
AI: [Searches articles] "Here's what I found about ChatGPT..."
```

## The API

**NewsAPI** - Get free key at https://newsapi.org/register
- Free tier: 100 requests/day
- Access to 80,000+ news sources

## Three Functions to Implement

### Function 1: `get_today_headlines(category, country)`

**What it does:** Fetches today's top breaking news

**When to use:**
- User asks about "today", "latest", "current" news
- No specific topic, just wants overview
- Examples: "What's happening in tech?", "Show me today's science news"

**Parameters:**
- `category`: "technology", "science", "business", "general"
- `country`: "us", "gb", "ca" (optional)

---

### Function 2: `search_articles(query, from_date, sort_by)`

**What it does:** Searches for articles about specific topics/companies/people

**When to use:**
- User mentions specific company (OpenAI, Google, Meta)
- User mentions specific technology (ChatGPT, transformers)
- User mentions specific person (Sam Altman, Elon Musk)
- User wants news from time period ("last week")
- Examples: "Find news about OpenAI", "What's happening with GPT-4?"

**Parameters:**
- `query`: Keywords to search
- `from_date`: "YYYY-MM-DD" format (optional, defaults to 7 days ago)
- `sort_by`: "publishedAt", "relevancy", "popularity" (optional)

---

### Function 3: `get_sources(category, country)`

**What it does:** Lists available news sources/publications

**When to use:**
- User asks what sources are available
- Examples: "What tech news sources do you have?", "Which publications cover science?"

**Parameters:**
- `category`: "technology", "science", "business", "general"
- `country`: "us", "gb", "ca" (optional)

## Decision Guide
```
Is user asking about sources?
    YES → get_sources()
    
Does user mention SPECIFIC topic/company/person?
    YES → search_articles()
    
Does user want general/today's news?
    YES → get_today_headlines()
```

**Quick Examples:**

| User Question | Function | Why |
|---------------|----------|-----|
| "What's new in AI?" | `get_today_headlines` | General, today's news |
| "Find articles about Meta" | `search_articles` | Specific company |
| "What news sources exist?" | `get_sources` | Asking about sources |
| "Show me news about Sam Altman from last week" | `search_articles` | Specific person + time |

## Test Cases

Your assistant should handle:

**Test 1:** "What's happening in tech today?"
- Calls: `get_today_headlines(category="technology")`

**Test 2:** "Find recent news about OpenAI"
- Calls: `search_articles(query="OpenAI")`

**Test 3:** "What tech news sources are available?"
- Calls: `get_sources(category="technology")`

**Test 4:** "Show me AI news from last week"
- Calls: `search_articles(query="AI", from_date="2024-11-04")`

**Test 5:** "What are today's science headlines?"
- Calls: `get_today_headlines(category="science")`


In [1]:
import os
import json
from openai import OpenAI
import requests
from datetime import datetime, timedelta

In [2]:
NEWSAPI_BASE_URL = "https://newsapi.org/v2"
GNEWS_API_BASE_URL = "https://gnews.io/api/v4"
NEWSAPI_API_KEY = "NEWSAPI_API_KEY"
GNEWS_API_KEY = "GNEWS_API_KEY"
OPENAI_API_KEY = "OPENAI_API_KEY"

def get_api_key(env_var):
    api_key = os.getenv(env_var)
    if not api_key:
        raise RuntimeError(f"Missing {env_var} key. Set the {env_var} secret.")
    return api_key

In [3]:
client = OpenAI(api_key = get_api_key(OPENAI_API_KEY))

In [4]:
def get_today_headlines(category, country, page_size = 5):
    api_key = get_api_key(NEWSAPI_API_KEY)

    params = {
        "apiKey": api_key,
        "category": category,
        "pageSize": page_size
    }

    if country:
        params["country"] = country

    url = f"{NEWSAPI_BASE_URL}/top-headlines"
    response = requests.get(url, params = params, timeout = 10)

    try:
        data = response.json()
    except Exception:
        raise Exception("NewsAPI did not return valid JSON")

    if response.status_code != 200 or data.get("status") != "ok":
        message = data.get("message", "Unknown error from NewsAPI")
        raise Exception(f"NewsAPI error: {message}")

    articles = data.get("articles", []) or []

    results = []
    for article in articles:
        results.append(
            {
                "title": article.get("title"),
                "description": article.get("description"),
                "url": article.get("url"),
                "source": article.get("source", {}).get("name"),
                "published_at": article.get("publishedAt")
            }
        )

    return results


def search_articles(query, from_date, sort_by = "publishedAt", page_size = 5):
    api_key = get_api_key(GNEWS_API_KEY)

    if from_date is None:
        seven_days_ago = datetime.utcnow() - timedelta(days = 7)
        from_date = seven_days_ago.strftime("%Y-%m-%d")

    params = {
        "apikey": api_key,
        "q": query,
        "from": from_date,
        "sortby": sort_by,
        "max": page_size,
        "lang": "en"
    }

    url = f"{GNEWS_API_BASE_URL}/search"

    response = requests.get(url, params = params, timeout = 10)

    try:
        data = response.json()
    except Exception:
        raise Exception("GNews did not return valid JSON")

    if response.status_code != 200 or "articles" not in data:
        message = data.get("errors", ["Unknown error from GNews"])[0]
        raise Exception(f"GNews error: {message}")

    articles = data.get("articles", []) or []

    results = []
    for article in articles:
        results.append(
            {
                "title": article.get("title"),
                "description": article.get("description"),
                "url": article.get("url"),
                "source": article.get("source", {}).get("name"),
                "published_at": article.get("published"),
            }
        )

    return results



def get_sources(category, country = None):
    api_key = get_api_key(NEWSAPI_API_KEY)

    params = {
        "apiKey": api_key,
        "category": category
    }

    if country:
        params["country"] = country

    url = f"{NEWSAPI_BASE_URL}/top-headlines/sources"
    response = requests.get(url, params = params, timeout = 10)

    try:
        data = response.json()
    except Exception:
        raise Exception("NewsAPI did not return valid JSON")

    if response.status_code != 200 or data.get("status") != "ok":
        message = data.get("message", "Unknown error from NewsAPI")
        raise Exception(f"NewsAPI error: {message}")

    sources = data.get("sources", []) or []

    results = []
    for src in sources:
        results.append(
            {
                "id": src.get("id"),
                "name": src.get("name"),
                "description": src.get("description"),
                "url": src.get("url"),
                "category": src.get("category"),
                "country": src.get("country"),
            }
        )

    return results

In [5]:
get_today_headlines_tool = {
    "type": "function",
    "function": {
        "name": "get_today_headlines",
        "description": "Fetch todays top news headlines for a given category and optional country.",
        "parameters": {
            "type": "object",
            "properties": {
                "category": {
                    "type": "string",
                    "description": "News category to fetch.",
                    "enum": ["technology", "science", "business", "general"],
                },
                "country": {
                    "type": "string",
                    "description": "Optional country filter. Use two letter code like us, gb, ca.",
                    "enum": ["us", "gb", "ca"],
                },
            },
            "required": ["category"],
        },
    },
}

search_articles_tool = {
    "type": "function",
    "function": {
        "name": "search_articles",
        "description": "Search for news articles about a specific topic, company, product, model, or person.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Keywords to search for, such as OpenAI, GPT-4, Sam Altman, or similar.",
                },
                "from_date": {
                    "type": "string",
                    "description": "Earliest publish date in YYYY-MM-DD format. Defaults to 7 days ago if not provided.",
                },
                "sort_by": {
                    "type": "string",
                    "description": "Sort order of the articles.",
                    "enum": ["publishedAt", "relevancy", "popularity"],
                },
            },
            "required": ["query"],
        },
    },
}

get_sources_tool = {
    "type": "function",
    "function": {
        "name": "get_sources",
        "description": "List available news sources for a given category and country.",
        "parameters": {
            "type": "object",
            "properties": {
                "category": {
                    "type": "string",
                    "description": "News category for which to list sources.",
                    "enum": ["technology", "science", "business", "general"],
                },
                "country": {
                    "type": "string",
                    "description": "Optional country filter. Use two letter code like us, gb, ca.",
                    "enum": ["us", "gb", "ca"],
                },
            },
            "required": ["category"],
        },
    },
}

tools = [get_today_headlines_tool, search_articles_tool, get_sources_tool]

In [6]:
system_prompt = """
You are an AI powered tech news assistant.

You have three tools available:

1. get_today_headlines(category, country)
   Use this tool when the user asks for todays, latest, current, or recent headlines, or a general overview of news in a category.
   If the user mentions tech, AI, or technology, use category="technology".
   If the user mentions science, use category="science".
   If the user mentions business or finance, use category="business".
   If the user does not specify a category, use category="general".
   If the user does not specify a country, use country="us".
   If the user clearly mentions a country that is not supported, explain that you only support us, gb, and ca for now.

2. search_articles(query, from_date, sort_by)
   Use this tool when the user asks about a specific topic, company, product, model, or person.
   Examples: OpenAI, Google, Meta, ChatGPT, GPT-4, Sam Altman, Elon Musk.
   If the user asks for news from a time range such as "from last week" or "from yesterday",
   convert that into a from_date string in YYYY-MM-DD format.
   If from_date is not provided, default to about 7 days ago.
   If the user does not specify sort order, use sort_by="publishedAt".

3. get_sources(category, country)
   Use this tool when the user asks which news sources or publications are available.
   If they ask about tech sources, use category="technology". If they ask about science sources, use category="science".
   If they do not specify, choose the closest category. If that is not clear, use category="general".
   If the user does not specify a country, use country="us".
   If they clearly mention a country that is not supported, explain that you only support us, gb, and ca for now.

Do not make up news. Do not try to answer from your knowledge without first making sure to get the correct and most up-to-date information from the tools.
Always prefer calling a tool instead of guessing.
"""

In [7]:
def _execute_tool(function_name, arguments):
    if function_name == "get_today_headlines":
        print("Using get_today_headlines tool...\n")
        return get_today_headlines(
            category = arguments.get("category"),
            country = arguments.get("country"),
        )

    if function_name == "search_articles":
        print("Using search_articles tool...\n")
        return search_articles(
            query = arguments.get("query"),
            from_date = arguments.get("from_date"),
            sort_by = arguments.get("sort_by") or "publishedAt"
        )

    if function_name == "get_sources":
        print("Using get_sources tool...\n")
        return get_sources(
            category = arguments.get("category"),
            country = arguments.get("country"),
        )

    return None


def _append_tool_messages(messages, tool_call, function_name, tool_result):
    messages.append(
        {
            "role": "assistant",
            "content": None,
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": function_name,
                        "arguments": tool_call.function.arguments,
                    },
                }
            ],
        }
    )

    messages.append(
        {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": json.dumps(tool_result),
        }
    )

In [8]:
def get_openai_answer(user_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = client.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages,
        tools = tools,
        tool_choice = "auto"
    )

    assistant_message = response.choices[0].message
    tool_calls = assistant_message.tool_calls

    if not tool_calls:
        return assistant_message.content or ""

    tool_call = tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments or "{}")

    tool_result = _execute_tool(function_name, arguments)
    if tool_result is None:
        return "I could not handle that request with the current tools."

    _append_tool_messages(messages, tool_call, function_name, tool_result)

    final_response = client.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages
    )

    return final_response.choices[0].message.content or ""

In [9]:
question = "what are the biggest news today in the US?\n"
print(question)
answer = get_openai_answer(question)
print(answer)

print("\n=================================================\n")

question = "show me the latest tech news\n"
print(question)
answer = get_openai_answer(question)
print(answer)

what are the biggest news today in the US?

Using get_today_headlines tool...

Here are some of the biggest news stories today in the US:

1. The Wall Street Journal reports on the looming fall of a Ukrainian city and what it signifies for Putin's war efforts. More details at The Wall Street Journal.

2. NPR covers a forecast about severe solar storms that could hit Earth Tuesday night, potentially triggering vibrant auroras visible across much of the northern US.

3. Variety shares that Jimmy Kimmel broke down in tears honoring the late bandleader Cleto Escobedo III on his show.

4. Politico reports allegations from an advocacy group that Venezuelans deported from the US to El Salvador were tortured, contradicting claims from the Trump administration.

5. The International Energy Agency discusses the urgent need for diversification and cooperation in the energy sector amid multiplying global risks.

If you want details or news on a specific topic, let me know!


show me the latest tec

In [10]:
question = "find recent articles about Rocket Lab\n"
print(question)
answer = get_openai_answer(question)
print(answer)

print("\n=================================================\n")

question = "search news about Sam Altman\n"
print(question)
answer = get_openai_answer(question)
print(answer)

find recent articles about Rocket Lab

Using search_articles tool...



C:\Users\Giorgi\AppData\Local\Temp\ipykernel_18096\1715401690.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  seven_days_ago = datetime.utcnow() - timedelta(days = 7)


Here are some recent articles about Rocket Lab:

1. "Rocket Lab delays debut of powerful, partially reusable Neutron rocket to 2026" - Rocket Lab has postponed the first launch of its medium-lift Neutron rocket to 2026. [Read more](https://www.space.com/space-exploration/rocket-lab-delays-debut-of-powerful-partially-reusable-neutron-rocket-to-2026) (Space.com)

2. "Rocket Lab, Planet Labs, And Archer Aviation Outshine Nvidia In 2025" - Rocket Lab is leading the aerospace industry, outperforming Nvidia and other AI chipmakers. [Read more](https://www.benzinga.com/markets/tech/25/11/48815509/space-beats-silicon-rocket-lab-planet-labs-and-archer-aviation-outshine-nvidia-in-2025) (Benzinga)

3. "What's Going On With Rocket Lab (RKLB) Stock Wednesday?" - Rocket Lab shares are moving bullishly after a strong third quarter. [Read more](https://www.benzinga.com/trading-ideas/movers/25/11/48808913/whats-going-on-with-rocket-lab-rklb-stock-wednesday-2) (Benzinga)

4. "What’s Going On With Rocket

In [11]:
question = "what tech news sources do you have?\n"
print(question)
answer = get_openai_answer(question)
print(answer)

print("\n=================================================\n")

question = "list science news publications in the us\n"
print(question)
answer = get_openai_answer(question)
print(answer)

what tech news sources do you have?

Using get_sources tool...

Here are some technology news sources available:

1. Ars Technica - The PC enthusiast's resource with a focus on power users and their tools. [Website](https://arstechnica.com)
2. Crypto Coins News - Breaking cryptocurrency news focusing on Bitcoin, Ethereum, ICOs, blockchain, and smart contracts. [Website](https://www.ccn.com)
3. Engadget - Daily coverage of new gadgets and consumer electronics. [Website](https://www.engadget.com)
4. Hacker News - Social news website focusing on computer science and entrepreneurship by Y Combinator. [Website](https://news.ycombinator.com)
5. Recode - Independent tech news, reviews, and analysis by respected journalists. [Website](http://www.recode.net)
6. TechCrunch - Leading media profiling startups and breaking tech news. [Website](https://techcrunch.com)
7. TechRadar - Latest technology news and reviews on computing, home entertainment, gadgets, and more. [Website](https://www.techrada